In [1]:
import geopandas as gpd
import pyproj
# Pyproj Correction
pyproj.datadir.set_data_dir('/Users/shai/anaconda3/envs/geo_env/share/proj')
import pandas as pd
import numpy as np
from shapely.geometry import Point
pd.options.display.max_columns =100
pd.options.display.max_rows =100


In [19]:
simRes = pd.read_excel('simulationResults_raw_april16.xlsx')

In [20]:
simRes.head()

,Unnamed: 0,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,stay,leave,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income
0,0,210_19,1,210_19_1,20180988,70,26,22,0,0,1,9431.0,1,0,d2252264-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0
1,1,210_19,2,210_19_2,20180988,70,30,38,0,0,1,12014.0,0,1,d22522fa-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0
2,2,210_19,3,210_19_3,20180988,70,15,44,0,0,1,13548.0,0,1,d2252322-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0
3,3,210_19,4,210_19_4,20180988,70,28,73,0,1,0,5644.0,0,1,d2252340-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,0,1,1,0,0
4,4,210_19,5,210_19_5,20180988,70,33,35,1,0,0,6760.0,1,0,d225235e-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,1,0,0


In [21]:
simRes.loc[(simRes['leave']==1) & (simRes['rent']==1),'LeavingRenters'] = 1
simRes.loc[(simRes['leave']==1) & (simRes['own']==1),'LeavingOwners'] = 1
simRes.loc[(simRes['New Comers']==1) & (simRes['rent']==1),'NewComersRenters'] = 1
simRes.loc[(simRes['New Comers']==1) & (simRes['own']==1),'NewComersOwners'] = 1


In [22]:
simRes['bldStaus'] = simRes['tic'].apply(
            lambda x: 'before' if x ==  0 else 'after')

In [23]:
AggByProject = simRes.groupby(['bldStaus','ProjNumber']).agg({'agentID':'count','bldCode':lambda x: x.nunique(),'Floors':lambda x: round(x.mean(),0),
                                                              'aprtmentSize':'mean','yearsInBldg':lambda x: round(x.mean(),1),
                               'age':lambda x: round(x.mean(),1),'income':lambda x: round(x.mean(),0),
                               'rent':'sum','own':'sum','LeavingRenters':'sum','LeavingOwners':'sum','NewComersRenters':'sum','NewComersOwners':'sum',
                               'prjectType':'first','tic':'first','CostForStaying':lambda x: round(x.mean(),0),
                                                              'ratioCostForStaying':lambda x: round(x.mean(),2),'stay':'sum','New Comers':'sum','leave':'sum',
                                                              'Under 65':'sum','Above 65':'sum','Low Income':'sum','Medium Income':'sum',
                                                              'High Income':'sum','bldStaus':'first'})
titleConvert = {'agentID':'Total Households',
                'cldCode':'Number of Apartments',
                'Floors':'Avg Floors',
                'aprtmentSize':'Apartment Size Sq Meters',
                'yearsInBldg':'Years in Building',
                'age':'Average Age','income':'average income',
                'rent':'Nubmer of renters','own':'Number of owners','LeavingRenters':'Leaving Renters','LeavingOwners':'Leaving Owners','NewComersRenters':'New Renters','NewComersOwners':'New Owners',
               'prjectType':'Type of Project','tic':'Tic',
               'CostForStaying':'Cost For Staying',
               'ratioCostForStaying':'Ratio Cost For Staying','stay':'Staying','leave':'Leaving','bldStaus':'Before or After Renewal'}
AggByProject.rename(columns=titleConvert,inplace=True)

In [24]:
simRes.head(2)

,Unnamed: 0,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,stay,leave,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income,LeavingRenters,LeavingOwners,NewComersRenters,NewComersOwners,bldStaus
0,0,210_19,1,210_19_1,20180988,70,26,22,0,0,1,9431.0,1,0,d2252264-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,before
1,1,210_19,2,210_19_2,20180988,70,30,38,0,0,1,12014.0,0,1,d22522fa-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0,NaN,NaN,NaN,NaN,before


In [25]:
AggByProject = AggByProject.reset_index()

In [26]:
AggByProjectBeforeAfter = AggByProject.pivot(index='ProjNumber',columns='bldStaus')

In [27]:
AggByProjectBeforeAfter.to_excel('AggByProjectBeforeAfter_april_18.xlsx')

In [28]:
a = pd.read_excel('AggByProjectBeforeAfter_april_18.xlsx', header=[0,1])

In [29]:
a

Unnamed: 0_level_0 Total Households        bldCode        Avg Floors  \
             bldStaus            after before   after before      after   
0          ProjNumber              NaN    NaN     NaN    NaN        NaN   
1            20100133             40.0   16.0     1.0    1.0        6.0   
2            20110270             48.0   18.0     1.0    1.0        7.0   
3            20120894             77.0   16.0     1.0    1.0        6.0   
4            20140060             40.0   15.0     1.0    1.0        7.0   
5            20140231             34.0    6.0     1.0    1.0        9.0   
6            20140467             41.0   16.0     1.0    1.0        7.0   
7            20140556             45.0   16.0     1.0    1.0        7.0   
8            20141182             40.0   12.0     1.0    1.0        3.0   
9            20150274             50.0   12.0     1.0    1.0        3.0   
10           20150857             47.0   12.0     1.0    1.0        8.0   
11           20160319             62.0   33.0     1.0    1.0        6.0   
12           20170076            112.0    4.0     3.0    1.0       10.0   
13           20171974             54.0   13.0     1.0    1.0       10.0   
14           20180345             76.0   39.0     1.0    1.0        8.0   
15           20180812             47.0   17.0     1.0    1.0        7.0   
16           20180845             48.0   22.0     1.0    1.0        7.0   
17           20180988             56.0   20.0     1.0    1.0        7.0   
18           20181092             45.0   16.0     1.0    1.0        7.0   
19           20181295             71.0   24.0     1.0    1.0        7.0   
20        502-0147678            232.0   12.0     2.0    1.0       18.0   
21        502-0178285            293.0   84.0     2.0    3.0       34.0   
22        502-0189720            207.0   27.0     3.0    3.0       20.0   
23        502-0196659            712.0  200.0     4.0   10.0       30.0   
24        502-0201483            234.0   17.0     2.0    1.0       15.0   
25        502-0316588            193.0   38.0     3.0    2.0       16.0   
26        502-0316752             72.0    2.0     2.0    2.0        6.0   
27        502-0317495            269.0   70.0     2.0    5.0       28.0   
28        502-0368860            153.0   29.0     1.0    1.0       32.0   
29        502-0424911            115.0   18.0     2.0    2.0       13.0   
30        502-0583948             78.0    9.0     2.0    1.0       11.0   
31        502-0654715            337.0   64.0     3.0    2.0       24.0   

          Apartment Size Sq Meters        Years in Building         \
   before                    after before             after before   
0     NaN                      NaN    NaN               NaN    NaN   
1     0.0                     82.0   70.0              11.8   29.6   
2     0.0                     92.0   80.0              10.4   27.8   
3     0.0                     72.0   60.0               5.0   23.9   
4     0.0                     82.0   70.0              10.0   26.5   
5     0.0                    115.0   90.0               1.9   10.7   
6     0.0                    102.0   90.0               9.5   24.4   
7     0.0                     82.0   70.0              10.7   30.0   
8     0.0                    112.0  100.0               7.0   23.5   
9     0.0                     92.0   80.0               8.1   33.9   
10    0.0                    115.0   90.0               7.3   28.4   
11    0.0                    102.0   90.0              13.0   24.5   
12    0.0                     75.0   50.0               0.2    6.8   
13    0.0                     85.0   60.0               3.6   15.2   
14    0.0                     92.0   80.0              13.1   25.5   
15    0.0                     62.0   50.0               6.7   18.6   
16    0.0                     98.0   86.0               6.5   14.2   
17    0.0                     82.0   70.0               7.1   19.8   
18    0.0                     92.0   80.0           

In [53]:
AggByProject

,bldStaus,ProjNumber,Total Households,bldCode,Avg Floors,Apartment Size Sq Meters,Years in Building,Average Age,average income,Nubmer of renters,Number of owners,Type of Project,Tic,Cost For Staying,Ratio Cost For Staying,Staying,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income,Before or After Renewal
0,after,20100133,40,40,6.000000,82,11.850000,43.950000,15291.476316,16,24,1,2,4334.285000,0.242399,2,24,32,8,11,10,19,after
1,after,20110270,48,48,7.000000,92,10.416667,50.750000,17147.323465,15,33,1,17,5005.247500,0.250573,3,30,35,13,12,8,28,after
2,after,20120894,77,77,6.000000,72,4.974026,47.233766,16484.183185,27,50,1,14,4846.674286,0.276248,9,61,63,14,5,61,11,after
3,after,20140060,40,40,7.000000,82,9.950000,47.700000,16404.130263,14,26,1,16,4503.160000,0.239276,5,25,31,9,6,17,17,after
4,after,20140231,34,34,9.000000,115,1.882353,45.705882,23819.250774,12,22,2,12,7886.185294,0.310445,0,28,28,6,5,1,28,after
5,after,20140467,41,41,7.000000,102,9.512195,38.780488,18646.797176,12,29,1,18,5431.616098,0.250776,2,25,37,4,8,8,25,after
6,after,20140556,45,45,7.000000,82,10.666667,53.244444,15533.474854,15,30,1,7,4621.660000,0.257775,1,29,30,15,15,10,20,after
7,after,20141182,40,40,3.000000,112,7.050000,43.400000,20642.505263,16,24,1,20,6551.485000,0.278165,2,28,30,10,6,6,28,after
8,after,20150274,50,50,3.000000,92,8.140000,44.640000,19514.794737,16,34,1,13,5848.740000,0.274600,5,38,42,8,3,12,35,after
9,after,20150857,47,47,8.000000,115,7.255319,47.042553,22751.975924,14,33,2,10,7197.641489,0.285357,4,35,35,12,6,6,35,after


In [32]:
AggByBuildings = simRes.groupby('bldCode').agg({'agentID':'count','aprtmentSize':'mean','yearsInBldg':'mean',
                               'age':'mean','income':'mean',
                               'rent':'sum','own':'sum',
                               'prjectType':'first','tic':'first','CostForStaying':'mean','ratioCostForStaying':'mean','stay':'sum','New Comers':'sum','Under 65':'sum','Above 65':'sum','Low Income':'sum','Medium Income':'sum','High Income':'sum','bldStaus':'first'})
titleConvert = {'agentID':'Households In Building',
                'aprtmentSize':'Apartment Size Sq Meters',
                'yearsInBldg':'Years in Building',
                'age':'Average Age','income':'average income',
                'rent':'Nubmer of renters','own':'Number of owners',
               'prjectType':'Type of Project','tic':'Tic',
               'CostForStaying':'Cost For Staying',
               'ratioCostForStaying':'Ratio Cost For Staying','stay':'Staying','bldStaus':'Before or After Renewal'}
AggByBuildings.rename(columns=titleConvert,inplace=True)

In [33]:
simRes.head()

,Unnamed: 0,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,stay,leave,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income,bldStaus
0,0,210_19,1,210_19_1,20180988,70,26,22,0,0,1,9431.0,1,0,d2252264-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0,before
1,1,210_19,2,210_19_2,20180988,70,30,38,0,0,1,12014.0,0,1,d22522fa-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0,before
2,2,210_19,3,210_19_3,20180988,70,15,44,0,0,1,13548.0,0,1,d2252322-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,0,1,0,before
3,3,210_19,4,210_19_4,20180988,70,28,73,0,1,0,5644.0,0,1,d2252340-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,0,1,1,0,0,before
4,4,210_19,5,210_19_5,20180988,70,33,35,1,0,0,6760.0,1,0,d225235e-9e00-11eb-8aa1-acde48001122,0,0,stay,0,0,0.0,0.0,0.0,0,0,0,1,0,0,1,0,1,0,0,before


In [34]:
AggByBuildings.reset_index(inplace=True)

In [36]:
AggByBuildings

,bldCode,Households In Building,Apartment Size Sq Meters,Years in Building,Average Age,average income,Nubmer of renters,Number of owners,Type of Project,Tic,Cost For Staying,Ratio Cost For Staying,Staying,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income,Before or After Renewal
0,140_1_rr,56,93,0.000000,48.535714,22922.416353,22,34,3,30,7701.882857,0.337336,0,56,41,15,0,0,56,after
1,201_7,9,80,32.777778,47.888889,7662.222222,5,4,0,0,0.000000,0.000000,9,0,7,2,4,5,0,before
2,201_75,29,87,20.655172,41.344828,10470.655172,4,25,0,0,0.000000,0.000000,29,0,24,5,10,19,0,before
3,201_75_rr,153,112,3.915033,47.915033,23749.201582,43,110,3,32,7679.996601,0.300699,6,124,121,32,10,19,124,after
4,201_7_rr,65,105,8.538462,48.323077,20680.296761,22,43,3,25,6527.042308,0.275631,3,47,51,14,8,10,47,after
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,225_6,28,60,30.250000,47.892857,2584.678571,12,16,0,0,0.000000,0.000000,28,0,21,7,28,0,0,before
97,225_6_rr,176,85,13.193182,47.278409,14390.266148,74,102,3,27,4133.487500,0.211011,25,92,136,40,52,49,75,after
98,225_8,28,60,27.035714,38.000000,8039.107143,12,16,0,0,0.000000,0.000000,28,0,25,3,12,16,0,before
99,225_8_rr,117,85,0.000000,45.743590,21354.213900,46,71,3,27,7083.404701,0.333174,0,117,99,18,0,14,103,after


In [37]:
columnsToPercent = ['Nubmer of renters','Number of owners','Staying','New Comers','Under 65','Above 65','Low Income','Medium Income','High Income']

In [38]:
columnsToPercent2 =[item+' Percent' for item in columnsToPercent]

In [39]:
columnsToPercent2

['Nubmer of renters Percent',
 'Number of owners Percent',
 'Staying Percent',
 'New Comers Percent',
 'Under 65 Percent',
 'Above 65 Percent',
 'Low Income Percent',
 'Medium Income Percent',
 'High Income Percent']

In [40]:
for i in range(len(columnsToPercent)):
    col = columnsToPercent[i]
    colPer = columnsToPercent2[i]
    AggByBuildings[colPer] = AggByBuildings[col]/AggByBuildings['Households In Building']

In [42]:
AggByBuildings

,bldCode,Households In Building,Apartment Size Sq Meters,Years in Building,Average Age,average income,Nubmer of renters,Number of owners,Type of Project,Tic,Cost For Staying,Ratio Cost For Staying,Staying,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income,Before or After Renewal,Nubmer of renters Percent,Number of owners Percent,Staying Percent,New Comers Percent,Under 65 Percent,Above 65 Percent,Low Income Percent,Medium Income Percent,High Income Percent
0,140_1_rr,56,93,0.000000,48.535714,22922.416353,22,34,3,30,7701.882857,0.337336,0,56,41,15,0,0,56,after,0.392857,0.607143,0.000000,1.000000,0.732143,0.267857,0.000000,0.000000,1.000000
1,201_7,9,80,32.777778,47.888889,7662.222222,5,4,0,0,0.000000,0.000000,9,0,7,2,4,5,0,before,0.555556,0.444444,1.000000,0.000000,0.777778,0.222222,0.444444,0.555556,0.000000
2,201_75,29,87,20.655172,41.344828,10470.655172,4,25,0,0,0.000000,0.000000,29,0,24,5,10,19,0,before,0.137931,0.862069,1.000000,0.000000,0.827586,0.172414,0.344828,0.655172,0.000000
3,201_75_rr,153,112,3.915033,47.915033,23749.201582,43,110,3,32,7679.996601,0.300699,6,124,121,32,10,19,124,after,0.281046,0.718954,0.039216,0.810458,0.790850,0.209150,0.065359,0.124183,0.810458
4,201_7_rr,65,105,8.538462,48.323077,20680.296761,22,43,3,25,6527.042308,0.275631,3,47,51,14,8,10,47,after,0.338462,0.661538,0.046154,0.723077,0.784615,0.215385,0.123077,0.153846,0.723077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,225_6,28,60,30.250000,47.892857,2584.678571,12,16,0,0,0.000000,0.000000,28,0,21,7,28,0,0,before,0.428571,0.571429,1.000000,0.000000,0.750000,0.250000,1.000000,0.000000,0.000000
97,225_6_rr,176,85,13.193182,47.278409,14390.266148,74,102,3,27,4133.487500,0.211011,25,92,136,40,52,49,75,after,0.420455,0.579545,0.142045,0.522727,0.772727,0.227273,0.295455,0.278409,0.426136
98,225_8,28,60,27.035714,38.000000,8039.107143,12,16,0,0,0.000000,0.000000,28,0,25,3,12,16,0,before,0.428571,0.571429,1.000000,0.000000,0.892857,0.107143,0.428571,0.571429,0.000000
99,225_8_rr,117,85,0.000000,45.743590,21354.213900,46,71,3,27,7083.404701,0.333174,0,117,99,18,0,14,103,after,0.393162,0.606838,0.000000,1.000000,0.846154,0.153846,0.000000,0.119658,0.880342


In [44]:
AggByBuildings.to_excel('aggbyBldgs_april_17_2021.xlsx')

In [ ]:
AggByBuildings

In [17]:
AggByBuildingsAfter = AggByBuildings[AggByBuildings['Before or After Renewal']=='after'].copy()

In [18]:
pointsAfter = gpd.read_file('PointAfter3DApril15_1.geojson')

In [19]:
pointsAfter.head()

,fid,Area,floors,height,bld_num,bld_addres,hebrew_adr,project_nu,units,project_ty,status,end_date,geometry
0,73,540.06174,7.0,25.0,95.0,215_24_a,"סוקולוב 24, בת ים",20110270,32.0,1.0,Building after,2100-12-23,POINT Z (175740.087 658954.522 6.000)
1,73,540.06174,7.0,25.0,95.0,215_24_a,"סוקולוב 24, בת ים",20110270,32.0,1.0,Building after,2100-12-23,POINT Z (175735.755 658957.492 6.000)
2,73,540.06174,7.0,25.0,95.0,215_24_a,"סוקולוב 24, בת ים",20110270,32.0,1.0,Building after,2100-12-23,POINT Z (175731.988 658963.825 6.000)
3,73,540.06174,7.0,25.0,95.0,215_24_a,"סוקולוב 24, בת ים",20110270,32.0,1.0,Building after,2100-12-23,POINT Z (175736.192 658969.584 6.000)
4,73,540.06174,7.0,25.0,95.0,215_24_a,"סוקולוב 24, בת ים",20110270,32.0,1.0,Building after,2100-12-23,POINT Z (175744.981 658971.130 6.000)


In [20]:
AggByBuildingsAfter.head()

,bldCode,Households In Building,Apartment Size Sq Meters,Years in Building,Average Age,average income,Nubmer of renters,Number of owners,Type of Project,Tic,Cost For Staying,Ratio Cost For Staying,Staying,New Comers,Under 65,Above 65,Low Income,Medium Income,High Income,Before or After Renewal,Nubmer of renters Percent,Number of owners Percent,Staying Percent,New Comers Percent,Under 65 Percent,Above 65 Percent,Low Income Percent,Medium Income Percent,High Income Percent
0,140_1_rr,56,93,0.000000,48.535714,22922.416353,22,34,3,30,7701.882857,0.337336,0,56,41,15,0,0,56,after,0.392857,0.607143,0.000000,1.000000,0.732143,0.267857,0.000000,0.000000,1.000000
3,201_75_rr,153,112,3.915033,47.915033,23749.201582,43,110,3,32,7679.996601,0.300699,6,124,121,32,10,19,124,after,0.281046,0.718954,0.039216,0.810458,0.790850,0.209150,0.065359,0.124183,0.810458
4,201_7_rr,65,105,8.538462,48.323077,20680.296761,22,43,3,25,6527.042308,0.275631,3,47,51,14,8,10,47,after,0.338462,0.661538,0.046154,0.723077,0.784615,0.215385,0.123077,0.153846,0.723077
6,201_9_rr,50,105,0.000000,45.740000,25222.309474,16,34,3,25,8679.310000,0.345165,0,50,36,14,0,0,50,after,0.320000,0.680000,0.000000,1.000000,0.720000,0.280000,0.000000,0.000000,1.000000
8,208_12_a,71,92,8.056338,46.605634,17445.398073,22,49,1,15,5260.757183,0.262865,4,47,60,11,16,11,44,after,0.309859,0.690141,0.056338,0.661972,0.845070,0.154930,0.225352,0.154930,0.619718


In [21]:
copyCols = ['Households In Building','Apartment Size Sq Meters','Years in Building','Average Age','average income','Type of Project','Tic',
            'Cost For Staying','Ratio Cost For Staying','Before or After Renewal','Nubmer of renters Percent','Number of owners Percent','Staying Percent','New Comers Percent','Under 65 Percent','Above 65 Percent','Low Income Percent','Medium Income Percent','High Income Percent']

In [22]:
group1Cols =['Apartment Size Sq Meters','Years in Building','Average Age','average income','Type of Project','Tic',
            'Cost For Staying','Ratio Cost For Staying','Before or After Renewal']
group2Cols = ['Nubmer of renters Percent','Number of owners Percent','Staying Percent','New Comers Percent','Under 65 Percent','Above 65 Percent','Low Income Percent','Medium Income Percent','High Income Percent']
groupManipulation = ['Renters or Owners','Staying or new Comers','Above or below 65','Income Group']

In [23]:
for bldCode in AggByBuildingsAfter.loc[:,'bldCode']:
    refBldg = AggByBuildingsAfter.query(f'bldCode=="{bldCode}"')
    refValues1 = [refBldg[c].values.tolist() for c in group1Cols]
    
    #group2Cols = ['Nubmer of renters Percent','Number of owners Percent','Staying Percent','New Comers Percent','Under 65 Percent','Above 65 Percent','Low Income Percent','Medium Income Percent','High Income Percent']
    
    refValues2 = [refBldg[c].values[0].tolist() for c in group2Cols]
    ratioDict = dict(zip(group2Cols,refValues2))
    
    subset = pointsAfter.query(f'bld_addres=="{bldCode}"').index
    subsetLen = len(subset)
    for i in range(len(group1Cols)):
        #print(group1Cols[i])
        pointsAfter.loc[subset,group1Cols[i]] =refValues1[i][0]
    part1 = int(ratioDict['Nubmer of renters Percent']*subsetLen)
    pointsAfter.loc[subset[0:part1],'Renters or Owners']='Renters'
    pointsAfter.loc[subset[part1:],'Renters or Owners']='Owners'

    part1 = int(ratioDict['Staying Percent']*subsetLen)
    pointsAfter.loc[subset[0:part1],'Staying or new Comers']='Stay'
    pointsAfter.loc[subset[part1:],'Staying or new Comers']='New Comers'

    part1 = int(ratioDict['Under 65 Percent']*subsetLen)
    pointsAfter.loc[subset[0:part1],'Above or below 65']='Under 65'
    pointsAfter.loc[subset[part1:],'Above or below 65']='Above 65'

    part1 = int(ratioDict['Low Income Percent']*subsetLen)
    part2 = int(ratioDict['Medium Income Percent']*subsetLen)
    
    pointsAfter.loc[subset[0:part1],'Income Group']='Low Income'
    pointsAfter.loc[subset[part1:part2],'Income Group']='Medium Income'
    pointsAfter.loc[subset[part2:],'Income Group']='High Income'
#     rentersPercent = group2Cols['Nubmer of renters Percent']
#     len(subset)

In [25]:
pointsAfter.to_file('pointsAfterWithDetailsApril16_2324.geojson',driver='GeoJSON')